# Linear Regression using `sklearn`

This Jupyter Notebook is adapted from a [class](http://cs109.github.io/2014/) at Harvard University.

## Brief recap of linear regression

[Linear Regression](http://en.wikipedia.org/wiki/Linear_regression) is a method to model the relationship between a set of independent variables $X$ (also knowns as explantory variables, features, predictors) and a dependent variable $Y$.  This method assumes the relationship bewteen each predictor $X$ is linearly related to the dependent variable $Y$.  

$$ Y = \beta_0 + \beta_1 X + \epsilon$$

where $\epsilon$ is considered as an unobservable randome variable that adds noise to the linear relatiosnhip. 

* $\beta_0$ is the intercept of the linear model

* Multiple linear regression is when you have more than one independent variable
    * $X_1$, $X_2$, $X_3$, $\ldots$

$$ Y = \beta_0 + \beta_1 X_1 + \ldots + \beta_p X_p + \epsilon$$ 

* The model in linear regression is the *conditional mean* of $Y$ given the values in $X$ is expressed a linear function.  

$$ y = f(x) = E(Y | X = x)$$ 

![conditional mean](conditionalmean.png)
http://www.learner.org/courses/againstallodds/about/glossary.html

* The goal is to estimate the coefficients (e.g. $\beta_0$ and $\beta_1$). We represent the estimates of the coefficients with a "hat" on top of the letter.  

$$ \hat{\beta}_0, \hat{\beta}_1 $$

* Once you estimate the coefficients $\hat{\beta}_0$ and $\hat{\beta}_1$, you can use these to predict new values of $Y$

$$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x_1$$


* How do you estimate the coefficients? 
    * There are many ways to fit a linear regression model
    * The method called **least squares** is one of the most common

### Least squares
[Least squares](http://en.wikipedia.org/wiki/Least_squares) is a method of estimating the cofficients of a linear model by minimizing the difference between the following: 

$$ S = \sum_{i=1}^N r_i = \sum_{i=1}^N (y_i - (\beta_0 + \beta_1 x_i))^2 $$

where $N$ is the number of observations.  

We won't go into the mathematical details, but the least squares estimates $\hat{\beta}_0$ and $\hat{\beta}_1$ minimize the sum of the squared **residuals** $r_i = y_i - (\beta_0 + \beta_1 x_i)$ in the model (i.e. makes the difference bewteen the observed $y_i$ and linear model $\beta_0 + \beta_1 x_i$ as small as possible). 

### Purposes of linear regression

Given a dataset $X$ and $Y$, linear regression can be used to: 

* Build a predictive model to predict future values of $X_i$ without a $Y$ value.  
* Model the strength of the relationship between each dependent variable $X_i$ and $Y$
    * Sometimes not all $X_i$ will have a relationship with $Y$
    * Need to figure out which $X_i$ contributes most information to determine $Y$

In [ ]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

# special matplotlib argument for improved plots
from matplotlib import rcParams

#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['axes.prop_cycle'] = mpl.cycler(color=dark2_colors)
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'

## Boston housing dataset

The [Boston Housing data set](https://archive.ics.uci.edu/ml/datasets/Housing) contains information about the housing values in suburbs of Boston.  This dataset was originally taken from the StatLib library which is maintained at Carnegie Mellon University and is now available on the UCI Machine Learning Repository.

We'll use this dataset to predict housing prices based on other variables in the dataset.

### Load the Boston Housing data set from `sklearn`

This data set is available in the [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston) python library. That is how we'll access it in this PyNotebook.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
type(boston)

According to the sklearn documentation, a `Bunch` is a:

    Dictionary-like object, the interesting attributes are: ‘data’, the data to learn, ‘target’, the regression targets, and ‘DESCR’, the full description of the dataset.
    
We also can access things like `keys`, `feature_names` and `target_names`. 

In [ ]:
boston.keys()

In [ ]:
boston.data.shape

In [ ]:
# Print column names
print(boston.feature_names)

In [ ]:
# Print description of Boston housing data set
print(boston.DESCR)

## Explore the dataset

This will be much simpler if we convert the dataset to a `pandas` DataFrame.

In [ ]:
boston_df = pd.DataFrame(boston.data)
boston_df.head()

There aren't any column headings, so let's create some.

In [ ]:
boston_df.columns = boston.feature_names
boston_df.head()

Now we have a pandas DataFrame called `boston_df` containing data we can use to predict Boston housing prices. Let's create a variable called `PRICE` which will contain the prices. This information is contained in the `target` data. 

In [ ]:
print(boston.target.shape)

In [ ]:
boston_df['PRICE'] = boston.target
boston_df.head()

### Summary statistics

In [ ]:
boston_df.describe()

### Scatter plots

Let's look at some scatter plots for three variables: `CRIM`, `RM` and `PTRATIO`. 

What kind of relationship do you see? e.g. positive, negative?  linear? non-linear? 

In [ ]:
plt.scatter(boston_df.CRIM, boston_df.PRICE)
plt.xlabel("Per capita crime rate by town (CRIM)")
plt.ylabel("Housing Price")
plt.title("Relationship between CRIM and Price");

In [ ]:
plt.scatter(boston_df.RM, boston_df.PRICE)
plt.xlabel("Average number of rooms per dwelling (RM)")
plt.ylabel("Housing Price")
plt.title("Relationship between RM and Price");

In [ ]:
plt.scatter(boston_df.PTRATIO, boston_df.PRICE)
plt.xlabel("Pupil-to-Teacher Ratio (PTRATIO)")
plt.ylabel("Housing Price")
plt.title("Relationship between PTRATIO and Price");

### Histograms

In [ ]:
plt.hist(boston_df.CRIM)
plt.title("CRIM")
plt.xlabel("Crime rate per capita")
plt.ylabel("Frequencey");

In [ ]:
plt.hist(boston_df.PRICE)
plt.title('Housing Prices: $Y_i$')
plt.xlabel('Price')
plt.ylabel('Frequency');

## Using linear regression to predict housing prices

If we are interested in predicting price, we need to build some kind of model. Let's use linear regression to predict price using the 13 variables in the `boston_df` DataFrame.  

There are several ways of doing this in Python. One way of doing this is to use `sklearn`. 

In our example,

$Y$ = Boston housing prices (also called "target" data)

and

$X$ = all the other features (or independent variables)

which we will use to fit a linear regression model and predict Boston housing prices. 

We will use the least squares method as the way to estimate the coefficients.

In [ ]:
from sklearn.linear_model import LinearRegression

X = boston_df.drop('PRICE', axis = 1)
Y = boston_df['PRICE']

Create a LinearRegression object:

In [ ]:
lm = LinearRegression()
lm

#### What can you do with a LinearRegression object?

In [ ]:
# Look inside a linear regression object
# LinearRegression.<tab>

Main functions | Description
--- | --- 
`lm.fit()` | Fit a linear model
`lm.predit()` | Predict Y using the linear model with estimated coefficients
`lm.score()` | Returns the coefficient of determination ( $R^2$ ) - *a measure of how well observed outcomes are replicated by the model, as the proportion of total variation of outcomes explained by the model*

#### What output can you get?

In [ ]:
# Look inside lm object
# lm.<tab>

Output | Description
--- | --- 
`lm.coef_` | Estimated coefficients
`lm.intercept_` | Estimated intercept 

### Fit a linear model

The `lm.fit()` function estimates the cofficients of the linear regression using least squares.

Use all 13 predictors to fit a linear regression model:

In [ ]:
lm.fit(X, Y)

Notice `fit_intercept=True` and `normalize=False`.

How would you change the model to fit without an intercept term?
Hint: See the [sklearn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

### Estimated intercept and coefficients

Let's look at the estimated coefficients from the linear model using `1m.intercept_` and `lm.coef_`.  

After we have fit our linear regression model using the least squares method, we want to see what are the estimates of our coefficients $\beta_0$, $\beta_1$, ..., $\beta_{13}$: 

$$ \hat{\beta}_0, \hat{\beta}_1, \ldots, \hat{\beta}_{13} $$

**Note**: The "hat" means it is an estimate of the coefficient.  

In [ ]:
print('Estimated intercept coefficient:', lm.intercept_)

In [ ]:
print('Number of coefficients:', len(lm.coef_))

#### The estimated coefficients:

In [ ]:
coeffs = list(map(list, zip(X.columns, lm.coef_)))
pd.DataFrame(coeffs, columns = ['features', 'estimatedCoefficients'])

### Predict prices 

We can calculate the predicted prices ($\hat{Y}_i$) using `lm.predict`. 

$$ \hat{Y}_i = \hat{\beta}_0 + \hat{\beta}_1 X_1 + \ldots \hat{\beta}_{13} X_{13} $$

Here are the first five predicted prices:

In [ ]:
lm.predict(X)[0:5]

In [ ]:
plt.hist(lm.predict(X))
plt.title('Predicted Housing Prices (fitted values): $\hat{Y}_i$')
plt.xlabel('Price')
plt.ylabel('Frequency');

### Plot the true prices compared to the predicted prices to see they whether they agree

In [ ]:
plt.scatter(boston_df.PRICE, lm.predict(X))
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted Prices: $Y_i$ vs $\hat{Y}_i$");

### Residual sum of squares (RSS)

$$ RSS = \sum_{i=1}^N r_i^2 = \sum_{i=1}^N (y_i - (\hat\beta_0 + \hat\beta_1 x_i))^2 $$

In [ ]:
np.sum((boston_df.PRICE - lm.predict(X)) ** 2)

### Mean squared error

In [ ]:
np.mean((boston_df.PRICE - lm.predict(X)) ** 2)

## Relationship between `PTRATIO` and housing price

Try fitting a linear regression model using only the `PTRATIO` (pupil-teacher ratio by town).

Calculate the mean squared error. 

In [ ]:
lm = LinearRegression()
lm.fit(X[['PTRATIO']], boston_df.PRICE)

In [ ]:
np.mean((boston_df.PRICE - lm.predict(X[['PTRATIO']])) ** 2)

### Plot the fitted linear regression line

In [ ]:
plt.scatter(boston_df.PTRATIO, boston_df.PRICE)
plt.xlabel("Pupil-to-Teacher Ratio (PTRATIO)")
plt.ylabel("Housing Price")
plt.title("Relationship between PTRATIO and Price")

plt.plot(boston_df.PTRATIO, lm.predict(X[['PTRATIO']]), color='blue', linewidth=3);

## Your turn

Try fitting a linear regression model using three independent variables:

1. `CRIM` (per capita crime rate by town)
2. `RM` (average number of rooms per dwelling)
3. `PTRATIO` (pupil-teacher ratio by town)

Calculate the mean squared error. 

In [ ]:
# Your turn

## Training and Validation datasets

Now let's build a linear model on a training data set and then test how well it performs on a test data set.

One way of doing this is to create training and testing datasets manually. 

In [ ]:
X_train = X[:-50]
X_test = X[-50:]
Y_train = boston_df.PRICE[:-50]
Y_test = boston_df.PRICE[-50:]
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape);

Another way to create training and testing sets is to split the data into random subsets using the function `train_test_split` in `sklearn.cross_validation`. 

In [ ]:
# Let's look at the function in the help file
#sklearn.cross_validation.train_test_split?

In [ ]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(
    X, boston_df.PRICE, test_size=0.33, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

Now let's build a linear regression model using our new training dataset.

In [ ]:
lm = LinearRegression()
lm.fit(X_train, Y_train)
pred_train = lm.predict(X_train)
pred_test = lm.predict(X_test)

Now calculate the mean squared error using just the test data and compare to mean squared from using all the data to fit the model.

In [ ]:
print("Fit a model X_train, and calculate MSE with Y_train:", np.mean((Y_train - lm.predict(X_train)) ** 2))
print("Fit a model X_train, and calculate MSE with X_test, Y_test:", np.mean((Y_test - lm.predict(X_test)) ** 2))

### Residual plots

In [ ]:
plt.scatter(lm.predict(X_train), lm.predict(X_train) - Y_train, c='b', s=40, alpha=0.5)
plt.scatter(lm.predict(X_test), lm.predict(X_test) - Y_test, c='g', s=40)
plt.hlines(y = 0, xmin=0, xmax = 50)
plt.title('Residual Plot using training (blue) and test (green) data')
plt.ylabel('Residuals');